# Confusion Scheme on MNIST

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import discriminant_analysis
from keras.datasets import mnist
from scipy.signal import argrelextrema
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


## Data preparation

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

number_1 = 7
number_2 = 0

## Train set
x_train = x_train[(y_train == number_1) | (y_train == number_2)]
y_train = y_train[(y_train == number_1) | (y_train == number_2)]

X_m = x_train.astype(float)
y_m = y_train.astype(float)

X_m = X_m.reshape(X_m.shape[0], X_m.shape[1]**2)

#X_m = X_m[1:5000,:] # we truncate the data to have a faster program. Feel free to test with hole database
#y_m = y_m[1:5000]

## Test set
x_test = x_test[(y_test == number_1) | (y_test == number_2)]
y_test = y_test[(y_test == number_1) | (y_test == number_2)]

x_test = x_test.astype(float)
y_test = y_test.astype(float)

x_test = x_test.reshape(x_test.shape[0], x_test.shape[1]**2)

## Linear Discriminatn Analysis (LDA)

Projection on to the first 2 linear discriminant components

In [3]:
print("Computing Linear Discriminant Analysis projection")
# Train
X2 = X_m.copy()
X2.flat[::X_m.shape[1] + 1] += 0.01  # Make X invertible
X_lda = discriminant_analysis.LinearDiscriminantAnalysis(n_components=2).fit_transform(X2, y_m)

plt.figure()
plt.title('Linear Discriminant Analysis')
plt.scatter(X_lda[(y_m == number_1)], y_m[y_m == number_1]) 
plt.scatter(X_lda[(y_m == number_2)], y_m[y_m == number_2]) 

# Test
X3 = x_test.copy()
X3.flat[::x_test.shape[1] + 1] += 0.01  # Make X invertible
X_lda_test = discriminant_analysis.LinearDiscriminantAnalysis(n_components=2).fit_transform(X3, y_test)

Computing Linear Discriminant Analysis projection


/home/gmunoz/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


##